In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

with open("setting.yaml") as yaml_file:

    SETTING = yaml.load(yaml_file)

PATH = make_path_dict(SETTING)

In [ ]:
gene_x_sample = pd.read_csv(PATH["gene_x_sample.processed.tsv"], sep="\t", index_col=0)

target_x_sample = pd.read_csv(PATH["target_x_sample.tsv"], sep="\t", index_col=0)

gene_sets = ccal.read_gmts(SETTING["gene_set_file_paths"])

gene_set_x_information = pd.read_csv(
    PATH["gene_set_x_information.tsv"], sep="\t", index_col=0
)

In [ ]:
for target_name, target_values in target_x_sample.iterrows():

    output_directory_path = "{}/{}".format(PATH["gsea/"], target_name)

    ccal.establish_path(output_directory_path, "directory")

    match_function_name = "compute_log_ratio"

    score_moe_p_value_fdr = pd.read_csv(
        "{}/{}/all.{}.tsv".format(
            PATH["find_differentially_expressed_gene/"],
            target_name,
            match_function_name,
        ),
        sep="\t",
        index_col=0,
    )

    gene_score = score_moe_p_value_fdr["Score"]

    gene_score = (gene_score - gene_score.mean()) / gene_score.std()

    gene_set_score = (
        ccal.single_sample_gseas(
            gene_score.to_frame(),
            gene_sets.loc[
                gene_set_x_information.index[
                    gene_set_x_information["Good"].astype(bool)
                ]
            ],
            statistic=SETTING["gsea_statistic"],
            n_job=SETTING["n_job"],
        )
        .squeeze()
        .sort_values()
    )

    gene_set_score.to_csv(
        "{}/{}.gene_set_gsea_score.tsv".format(
            output_directory_path, match_function_name
        ),
        sep="\t",
        header=True,
    )

    print(gene_set_x_information.loc[gene_set_score.index])

    ccal.plot_rank_and_highlight(
        gene_set_score,
        SETTING["gene_sets_to_peek"],
        title=dict(text="{}<br>Gene Set Rank and Highlight".format(target_name)),
        yaxis=dict(title="Score (GSEA)"),
        html_file_path="{}/{}.rank_and_highlight.html".format(
            output_directory_path, match_function_name
        ),
    )

    output_directory_path = "{}/mountain".format(output_directory_path)

    ccal.establish_path(output_directory_path, "directory")

    for gene_set_name in SETTING["gene_sets_to_peek"]:

        ccal.single_sample_gsea(
            gene_score,
            gene_sets.loc[gene_set_name],
            statistic=SETTING["gsea_statistic"],
            title="{}<br>{}".format(target_name, gene_set_name),
            html_file_path="{}/{}.html".format(output_directory_path, gene_set_name),
        )